In [1]:
# read in the credentials as environment variables
import sys
import os
cwd = os.getcwd()
if cwd.__contains__('src'):
    env_vars = !cat /src/.env
elif cwd.startswith('/data/users/jqhu'):
    env_vars = !cat /data/users/jqhu/work/tamagotchi/smartFlies/.env
    sys.path.append('/data/users/jqhu/work/tamagotchi/smartFlies')
else:
    env_vars = !cat /gscratch/walkerlab/jqhu/smartFlies/tamagotchi/.env

for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

from schemas.schema_v4 import *

2025-04-14 19:53:52.507854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744660432.521038  711037 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744660432.525340  711037 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-14 19:53:52.540970: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-04-14 19:53:54,271][INFO]: Connecting jq@dev0.uwcnc.net:3306
[2025-04-14 19:53:54,279][INFO]: Connected jq@dev0.uwcnc.n

In [2]:
# (TrainingConfig() & f'training_config_hash="074eef10d15f46153fe6433861b160b8"')
schema.jobs.fetch()

array([('__training_result', '36e8239c389225c8c7b4f038615a6287', 'error', {'training_config_hash': '7189836cfda4c64933e0bf46eb601d00'}, 'LostConnectionError: Connection was lost during a transaction.', 'Traceback (most recent call last):\n  File "/usr/local/lib/python3.10/dist-packages/datajoint/autopopulate.py", line 315, in _populate1\n    make(dict(key), **(make_kwargs or {}))\n  File "/src/tamagotchi/schemas/schema_v4.py", line 161, in make\n    self.insert1(key)\n  File "/usr/local/lib/python3.10/dist-packages/datajoint/table.py", line 349, in insert1\n    self.insert((row,), **kwargs)\n  File "/usr/local/lib/python3.10/dist-packages/datajoint/table.py", line 452, in insert\n    self.connection.query(\n  File "/usr/local/lib/python3.10/dist-packages/datajoint/connection.py", line 351, in query\n    raise errors.LostConnectionError(\ndatajoint.errors.LostConnectionError: Connection was lost during a transaction.\n', 'jq@198.48.92.26', 'g3110', 30310, 949, datetime.datetime(2025, 4,

In [92]:
key_to_train = {}
for entry in schema.jobs.fetch():
    key_hash = entry[1]
    status = entry[4]
    train_hash = entry[3]['training_config_hash']
    key_to_train[key_hash] = [train_hash, status]
    
# find in the training config table
for key_hash, tpl in key_to_train.items():
    train_hash = tpl[0]
    status = tpl[1]
    training_config = TrainingConfig()
    print(f"key_hash: {key_hash}, train_hash: {train_hash}")
    training_config = (training_config & f'training_config_hash="{train_hash}"')
    
    print(training_config.fetch('save_dir'), "seed:", training_config.fetch('seed'), status)


key_hash: 31c248f134188eb2422e46d5a4adbd4a, train_hash: 8dd986dd8bbea6f6ce9dba778e4bda14
['/src/data/wind_sensing/apparent_wind_visual_feedback/sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/'] seed: [18024] 
key_hash: 36e8239c389225c8c7b4f038615a6287, train_hash: 7189836cfda4c64933e0bf46eb601d00
['/src/data/wind_sensing/apparent_wind_visual_feedback/sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/'] seed: [30592] LostConnectionError: Connection was lost during a transaction.
key_hash: 3d8d1465ebd0dda284abceda72af4cf4, train_hash: a9c7366248686dbf3b49e94e52a672b5
['/src/data/wind_sensing/apparent_wind_visual_feedback/sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/'] seed: [7679] 
key_hash: 413ef07ede666961b2311bf5530b60a5, train_hash: 156f1c1afb9c1c55d560d5a01cbeb0d2
['/src/data/wind_sensing/apparent_wind_visual_feedback/sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/'] seed: [18028] LostConnectionError: Connection was lost during a transaction.
k

In [3]:
# (schema.jobs & 'connection_id="898"').delete()
schema.jobs
# (schema.jobs & 'key_hash="36e8239c389225c8c7b4f038615a6287"').fetch()
# '7189836cfda4c64933e0bf46eb601d00'

# (schema.jobs & 'status="reserved"')
# (schema.jobs & 'host="d37f9df0fffb"').delete()
# (schema.jobs & 'key_hash="c66f480230103ca9123cecd6b3f8df96"').delete()
# (schema.jobs & 'status="error"' & 'key_hash="0bb9a16627e73f4077ef6a5c2fbc67b9"').delete()
# (schema.jobs & 'status="error"' & 'key_hash="22877960269526f63370b8c034837d9c"').fetch()

# schema.jobs.delete()
# training_config_hash_to_delete = ["8171898c23c5a659f66124c1e971b1b4", "7d7cfda61322daa8d05d557dbc8fa2c7", "8a14ed9755f248b49c29679f7ac785fa"]
# corresponding_ket_hash = ["1f65538675f8123736bb137551933cd3", "708745d318ca005a594a6c86694dd4e5", "ba0c80ee557474bb7fe993cb0b96c126"]
# for h in corresponding_ket_hash:
#     print((schema.jobs & f'key_hash="{h}"'))
#     (schema.jobs & f'key_hash="{h}"').delete()


table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__training_result,36e8239c389225c8c7b4f038615a6287,error,=BLOB=,LostConnectionError: Connection was lost during a transaction.,=BLOB=,jq@198.48.92.26,g3110,30310,949,2025-04-12 00:31:00
__training_result,413ef07ede666961b2311bf5530b60a5,error,=BLOB=,LostConnectionError: Connection was lost during a transaction.,=BLOB=,jq@198.48.92.26,g3110,30309,949,2025-04-12 00:32:46
__training_result,5fdbb076cd8272bb379966f2aefffb14,error,=BLOB=,LostConnectionError: Connection was lost during a transaction.,=BLOB=,jq@10.158.30.81,d37f9df0fffb,102,842,2025-04-11 23:43:43
__training_result,60fc37e97f0cfd36dffcbd2c96d96bb9,error,=BLOB=,LostConnectionError: Connection was lost during a transaction.,=BLOB=,jq@198.48.92.26,g3110,30311,949,2025-04-11 23:26:46
__training_result,6cb72adb45c929d3ce0806d09e39aca5,error,=BLOB=,AttributeError: 'Namespace' object has no attribute 'mlflow',=BLOB=,jq@10.158.30.81,d37f9df0fffb,104,419,2025-04-11 02:05:43
__training_result,978523d6cef932ad5356f7c75d953e7c,error,=BLOB=,LostConnectionError: Connection was lost during a transaction.,=BLOB=,jq@10.158.30.81,d37f9df0fffb,103,842,2025-04-12 01:17:26
__training_result,a99e8baf779306de9f4604f0a2ac15a5,error,=BLOB=,AttributeError: 'Namespace' object has no attribute 'mlflow',=BLOB=,jq@10.158.30.81,d37f9df0fffb,102,419,2025-04-11 00:54:53
__training_result,bb0356b9d0431d85a6b954f93d7ae300,error,=BLOB=,LostConnectionError: Connection was lost during a transaction.,=BLOB=,jq@10.158.30.81,d37f9df0fffb,104,842,2025-04-11 23:59:13


In [ ]:
TrainingConfig()
(TrainingConfig() & 'save_dir="/src/data/wind_sensing/apparent_wind_visual_feedback/dt0.1_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind"')

# TrainingConfig().fetch('save_dir')

# for hash in ['c97448325b629989b2d82e8f2f325c55', '45513dd8ac9d9cdbb3a34f957436f7af', 'dee781bba1737b60f0e9b494f6acfdfc']:
#     print((TrainingConfig() & f'training_config_hash="{hash}"').fetch()[['save_dir', 'seed']])
# (TrainingConfig() & 'training_config_hash="d1cc1b54c140f764025f8fc169a34456"').fetch()[['save_dir', 'seed']]

training_config_hash,algo,lr,eps,alpha,gamma,use_gae,gae_lambda,entropy_coef,value_loss_coef,max_grad_norm,seed,cuda_deterministic,num_processes,num_steps,ppo_epoch,num_mini_batch,clip_param,log_interval,save_interval,no_cuda,use_proper_time_limits,recurrent_policy,use_linear_lr_decay,env_name,log_dir,save_dir,outsuffix,dynamic,eval_type,eval_episodes,eval_interval,weight_decay,rnn_type,hidden_size,betadist,stacking,masking,stride,dataset,num_env_steps,qvar,birthx,diff_max,diff_min,birthx_linear_tc_steps,birthx_max,dryrun,curriculum,turnx,movex,auto_movex,auto_reward,loc_algo,time_algo,env_dt,walking,radiusx,diffusion_min,diffusion_max,r_shaping,wind_rel,action_feedback,squash_action,flipping,odor_scaling,stray_max,test_episodes,viz_episodes,model_fname,obs_noise,act_noise,cuda,apparent_wind,apparent_wind_allo,visual_feedback,if_vec_norm,if_train_actor_std
0cbffe67a5336f12bc3d3e0ccdca7aed,ppo,0.0003,1e-05,0.99,0.99,1,0.95,0.005,0.5,0.5,8400,0,4,2048,10,4,0.2,1.0,100.0,0,0,1,1,plume,/src/data/wind_sensing/apparent_wind_visual_feedback/logs/,/src/data/wind_sensing/apparent_wind_visual_feedback/dt0.1_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/,8400_0cbffe67a5336f12bc3d3e0ccdca7aed,0,skip,20,None,0.0001,VRNN,64,0,0,None,1,=BLOB=,10000000,=BLOB=,0.01,=BLOB=,=BLOB=,7,1.0,0,0,1.0,1.0,0,0,uniform,uniform,0.1,0,1.0,1.0,1.0,=BLOB=,1,0,1,1,1,2.0,50,10,,0.0,0.0,1,1,0,1,1,1
2b27687234e28c2f453ed45afa739884,ppo,0.0003,1e-05,0.99,0.99,1,0.95,0.005,0.5,0.5,7587,0,4,2048,10,4,0.2,1.0,100.0,0,0,1,1,plume,/src/data/wind_sensing/apparent_wind_visual_feedback/logs/,/src/data/wind_sensing/apparent_wind_visual_feedback/dt0.1_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/,7587_2b27687234e28c2f453ed45afa739884,0,skip,20,None,0.0001,VRNN,64,0,0,None,1,=BLOB=,10000000,=BLOB=,0.01,=BLOB=,=BLOB=,7,1.0,0,0,1.0,1.0,0,0,uniform,uniform,0.1,0,1.0,1.0,1.0,=BLOB=,1,0,1,1,1,2.0,50,10,,0.0,0.0,1,1,0,1,1,1
734fb3de55e2342206d3cb961b9fa7be,ppo,0.0003,1e-05,0.99,0.99,1,0.95,0.005,0.5,0.5,5471,0,4,2048,10,4,0.2,1.0,100.0,0,0,1,1,plume,/src/data/wind_sensing/apparent_wind_visual_feedback/logs/,/src/data/wind_sensing/apparent_wind_visual_feedback/dt0.1_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/,5471_734fb3de55e2342206d3cb961b9fa7be,0,skip,20,None,0.0001,VRNN,64,0,0,None,1,=BLOB=,10000000,=BLOB=,0.01,=BLOB=,=BLOB=,7,1.0,0,0,1.0,1.0,0,0,uniform,uniform,0.1,0,1.0,1.0,1.0,=BLOB=,1,0,1,1,1,2.0,50,10,,0.0,0.0,1,1,0,1,1,1
7c081cad98f9d9479d7352ddcd0a1bdf,ppo,0.0003,1e-05,0.99,0.99,1,0.95,0.005,0.5,0.5,31578,0,4,2048,10,4,0.2,1.0,100.0,0,0,1,1,plume,/src/data/wind_sensing/apparent_wind_visual_feedback/logs/,/src/data/wind_sensing/apparent_wind_visual_feedback/dt0.1_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/,31578_7c081cad98f9d9479d7352ddcd0a1bdf,0,skip,20,None,0.0001,VRNN,64,0,0,None,1,=BLOB=,10000000,=BLOB=,0.01,=BLOB=,=BLOB=,7,1.0,0,0,1.0,1.0,0,0,uniform,uniform,0.1,0,1.0,1.0,1.0,=BLOB=,1,0,1,1,1,2.0,50,10,,0.0,0.0,1,1,0,1,1,1
9f8387aa0eb922d6e04124ccbf50ecd5,ppo,0.0003,1e-05,0.99,0.99,1,0.95,0.005,0.5,0.5,3735,0,4,2048,10,4,0.2,1.0,100.0,0,0,1,1,plume,/src/data/wind_sensing/apparent_wind_visual_feedback/logs/,/src/data/wind_sensing/apparent_wind_visual_feedback/dt0.1_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/,3735_9f8387aa0eb922d6e04124ccbf50ecd5,0,skip,20,None,0.0001,VRNN,64,0,0,None,1,=BLOB=,10000000,=BLOB=,0.01,=BLOB=,=BLOB=,7,1.0,0,0,1.0,1.0,0,0,uniform,uniform,0.1,0,1.0,1.0,1.0,=BLOB=,1,0,1,1,1,2.0,50,10,,0.0,0.0,1,1,0,1,1,1
a4e49a805464c45d26151d528b4be653,ppo,0.0003,1e-05,0.99,0.99,1,0.95,0.005,0.5,0.5,27612,0,4,2048,10,4,0.2,1.0,100.0,0,0,1,1,plume,/src/data/wind_sensing/apparent_wind_visual_feedback/logs/,/src/data/wind_sensing/apparent_wind_visual_feedback/dt0.1_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/,27612_a4e49a805464c45d26151d528b4be653,0,skip,20,None,0.0001,VRNN,64,0,0,None,1,=BLOB=,10000000,=BLOB=,0.01,=BLOB=,=BLOB=,7,1.0,0,0,1.0,1.0,0,0,uniform,uniform,0.1,0,1.0,1.0,1.0,=

In [ ]:
# (TrainingConfig() & 'training_config_hash="8a14ed9755f248b49c29679f7ac785fa"')

training_config_hash,algo,lr,eps,alpha,gamma,use_gae,gae_lambda,entropy_coef,value_loss_coef,max_grad_norm,seed,cuda_deterministic,num_processes,num_steps,ppo_epoch,num_mini_batch,clip_param,log_interval,save_interval,no_cuda,use_proper_time_limits,recurrent_policy,use_linear_lr_decay,env_name,log_dir,save_dir,outsuffix,dynamic,eval_type,eval_episodes,eval_interval,weight_decay,rnn_type,hidden_size,betadist,stacking,masking,stride,dataset,num_env_steps,qvar,birthx,diff_max,diff_min,birthx_linear_tc_steps,birthx_max,dryrun,curriculum,turnx,movex,auto_movex,auto_reward,loc_algo,time_algo,env_dt,walking,radiusx,diffusion_min,diffusion_max,r_shaping,wind_rel,action_feedback,squash_action,flipping,odor_scaling,stray_max,test_episodes,viz_episodes,model_fname,obs_noise,act_noise,cuda,apparent_wind,apparent_wind_allo,visual_feedback,if_vec_norm,if_train_actor_std
8a14ed9755f248b49c29679f7ac785fa,ppo,0.0003,1e-05,0.99,0.99,1,0.95,0.005,0.5,0.5,15717,0,4,2048,10,4,0.2,1.0,100.0,0,0,1,1,plume,/tmp/gym/,/src/data/wind_sensing/apparent_wind_visual_feedback/sw_dist_logstep_wind_0.001_obs_noise_16deg_fix_actor_std/,15717_8a14ed9755f248b49c29679f7ac785fa,0,skip,20,None,0.0001,VRNN,64,0,0,None,1,=BLOB=,10000000,=BLOB=,0.001,=BLOB=,=BLOB=,7,1.0,0,0,1.0,1.0,0,0,uniform,uniform,0.04,0,1.0,1.0,1.0,=BLOB=,1,0,1,1,1,2.0,50,10,,16.0,0.0,1,1,0,1,0,0


In [5]:
# TrainingResult.populate(reserve_jobs=True)
# 8171898c23c5a659f66124c1e971b1b4
# 7d7cfda61322daa8d05d557dbc8fa2c7
# 8a14ed9755f248b49c29679f7ac785fa
# 
# TrainingResult().insert(
    
#     [dict(training_config_hash='8a14ed9755f248b49c29679f7ac785fa',
#         seed=15717,
#         hours_elapsed=0)]
#     , allow_direct_insert=True
# )

# TrainingResult() & 'training_config_hash="8a14ed9755f248b49c29679f7ac785fa"'
TrainingResult()

training_config_hash,hours_elapsed,seed
001935298c849ccde8dea2fa9d1f38b3,11.105126738879417,30698
00679d6cb443c109ececb7f0f86dd20e,15.257066301239862,1312
0c7a79ccdebe664752a33bcb3d117cac,15.714868382679091,8250
0d59ae1f145ef968c1fbca92b98fb320,9.420013713969125,24371
0d71853abf005913581425407573074c,11.597148582206833,25256
0daaa07e31a5b326ce3351d8a3c89578,10.653522274560398,11620
0fcef4af4850eceb7fd6c0c34d439693,14.916715725461643,30891
165c3dae9c1f9a429f3ed67ae5eb0fcc,10.102924041814275,22035
176735354319188f184f90f216a9281c,6.6522550606065325,21289
1973a995a6e651823bc2e8e0bdda9caf,16.330197767284183,19213


In [ ]:
training_config_hash = [item[0] for item in TrainingResult().fetch()]
training_config = [item[0] for item in TrainingConfig().fetch()]
to_be_done_exp = set(training_config).difference(set(training_config_hash))
print(len(to_be_done_exp))
undone = ['/plume_10057_5e0b837fa56b6a4dffff1a034adcff23.pt',
'/plume_11357_85f2b252e3fb3537cc8b97e7299543d1.pt',
'/plume_13356_edf288d268883f38671e8d8b3c6faa80.pt',
'/plume_19417_4ae9997221924be5c7af5b45ebc2f399.pt',
'/plume_23197_87f2d20ceaa4d4567390aef42c9a18b3.pt',
'/plume_23311_010d39e8723185121880b9cb06092819.pt',
'/plume_31287_f381b0d5c73c6a567dc6a53437118d1b.pt',
'/plume_6337_48886f495fca96cc3bd4e5fae0eff333.pt',
"/plume_8898_d8e5f14b5b73bcc1d45585b57d7f6b8c.pt"]

undone = [exp.split('_')[-1].replace('.pt', '') for exp in undone]
print(len(undone))
print([exp for exp in to_be_done_exp if exp in undone])
to_be_done_exp
for hash in to_be_done_exp:
    print((TrainingConfig() & f'training_config_hash="{hash}"').fetch()['seed'])
# (TrainingConfig() & 'training_config_hash="061f81f6e646a49a94eb2d27c0bf41ef"')

# insert a hyak-based example 061824

In [10]:
dict_to_insert = {
 'save_dir': '/src/data/wind_sensing/apparent_wind_visual_feedback/dt0.1_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/',
#  'outsuffix': '', # to be built by insert 
 'num_processes': 4, # nproc=8 for 4 concurrently agents. 
#  'seed': 2894, # to be sampled when inserting
 'dataset': ['constantx5b5', 'constant_mag_narrowx5b5', 'poisson_noisy3x5b5', 'poisson_mag_narrow_noisy3x5b5'],
 'num_env_steps': 10000000,
 'birthx': 0.01, # TODO: verify this is what Toha has
 'qvar': [2.0, 1.0, 0.5, 0.5], # Variance of init. location; higher = more off-plume initializations. Note this is set to 0 in evalCli 
 'diff_max': [0.8, 0.8, 0.8, 0.8],
 'diff_min': [0.7, 0.7, 0.4, 0.4],
 'apparent_wind': True,
 'apparent_wind_allo': False,
 'visual_feedback': True,
 'birthx_linear_tc_steps': 7, 
 'birthx_max': 1.0,
 'dryrun': False,
 'algo': 'ppo',
 'lr': 0.0003,
 'eps': 1e-05,
 'alpha': 0.99,
 'gamma': 0.99,
 'use_gae': True,
 'gae_lambda': 0.95,
 'entropy_coef': 0.005,
 'value_loss_coef': 0.5,
 'max_grad_norm': 0.5,
 'cuda_deterministic': False,
 'num_steps': 2048,
 'ppo_epoch': 10,
 'num_mini_batch': 4,
 'clip_param': 0.2,
 'log_interval': 1,
 'save_interval': 100,
 'no_cuda': False,
 'use_proper_time_limits': False,
 'recurrent_policy': True,
 'use_linear_lr_decay': True,
 'env_name': 'plume',
 'log_dir': '/src/data/wind_sensing/apparent_wind_visual_feedback/logs/',
 'dynamic': False,
 'eval_type': 'skip',
 'eval_episodes': 20,
 'eval_interval': None,
 'weight_decay': 0.0001,
 'rnn_type': 'VRNN',
 'hidden_size': 64,
 'betadist': False,
 'stacking': 0,
 'masking': None,
 'stride': 1,
 'curriculum': False,
 'turnx': 1.0,
 'movex': 1.0,
 'auto_movex': False,
 'auto_reward': False,
 'loc_algo': 'uniform',
 'time_algo': 'uniform',
 'env_dt': 0.1,
 'walking': False,
 'radiusx': 1.0,
 'diffusion_min': 1.0,
 'diffusion_max': 1.0,
 'r_shaping': ['step', 'oob'],
 'wind_rel': True,
 'action_feedback': False,
 'squash_action': True,
 'flipping': True,
 'odor_scaling': True,
 'stray_max': 2.0,
 'test_episodes': 50,
 'viz_episodes': 10,
 'model_fname': '',
 'obs_noise': 0.0, # 16deg = 0.28 rad
 'act_noise': 0.0,
 'cuda': True,
 'if_vec_norm': 1,
 'if_train_actor_std': True,
 }

In [11]:
for i in range(15):
    TrainingConfig.insert1(dict_to_insert)

No seed provided. Generating a new one.
No seed provided. Generating a new one.
No seed provided. Generating a new one.
No seed provided. Generating a new one.
No seed provided. Generating a new one.
No seed provided. Generating a new one.
No seed provided. Generating a new one.
No seed provided. Generating a new one.
No seed provided. Generating a new one.
No seed provided. Generating a new one.
No seed provided. Generating a new one.
No seed provided. Generating a new one.
No seed provided. Generating a new one.
No seed provided. Generating a new one.
No seed provided. Generating a new one.


In [12]:
TrainingConfig().fetch()['outsuffix']
len(TrainingConfig().fetch()['outsuffix'])


150

In [ ]:
schema.jobs

# filtering table entries

In [ ]:
TrainingConfig & 'lr < 1'
TrainingConfig().fetch()['save_dir']

# scaping the table

In [ ]:
TrainingConfig.drop()